In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from lassonet import LassoNetRegressor, plot_path, LassoNetRegressorCV

# --- 1. Data Generation ---
np.random.seed(42)
n_samples = 500
n_features = 10
n_informative = 2

X = np.random.rand(n_samples, n_features) * 10 - 5 # Features in [-5, 5]
y_true = np.sin(np.pi * X[:, 0] / 5) + (X[:, 1] / 3)**2
noise_level = 0.5
y = y_true + noise_level * np.random.randn(n_samples)
y = y.reshape(-1, 1) # Reshape y to be a column vector for LassoNet

# --- 2. Data Preprocessing ---
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y) # Also scale y for better NN performance

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_scaled, test_size=0.2, random_state=42
)
y_train = y_train.ravel() # LassoNetRegressor expects 1D y for path/fit
y_test = y_test.ravel()


In [2]:
# --- 3. Model Definition and Path Computation ---
print("Training LassoNetRegressor and computing path...")
model = LassoNetRegressor(
    hidden_dims=(64,),    # Single hidden layer with 64 neurons
    lambda_start='auto',  # Automatically determine starting lambda
    path_multiplier=1.02, # Controls how fast lambda decreases
    patience=10,          # For early stopping of internal epochs
    random_state=42,
    verbose=0             # Set to 1 for more output
)

# The.path() method computes the sequence of models
# return_state_dicts=True allows loading specific models from the path
path_info = model.path(X_train, y_train, X_val=X_test, y_val=y_test, return_state_dicts=True)

print(f"Path computed. Number of models in path: {len(path_info)}")


Training LassoNetRegressor and computing path...
Path computed. Number of models in path: 339


In [3]:
path_info

[HistoryItem(lambda_=0, state_dict={'layers.0.weight': tensor([[-1.3809e-01,  6.2122e-01, -2.9867e-01, -1.0135e-01, -2.8050e-02,
          -3.1084e-03, -2.6131e-02,  8.9289e-02,  1.2739e-01,  9.8224e-02],
         [-4.1805e-02,  2.4185e-02, -1.7471e-01,  2.7045e-02,  2.3817e-01,
           6.6647e-03, -2.2078e-01, -3.8969e-01,  1.3795e-01,  8.1057e-02],
         [ 1.3123e-01, -4.6777e-01,  1.6092e-02, -9.9532e-02, -1.4573e-01,
          -2.1536e-02, -3.2105e-02,  1.2002e-01, -1.1515e-01, -1.7367e-01],
         [-4.0197e-01, -1.3146e-01,  2.4896e-01, -1.4095e-01,  1.0001e-01,
           3.0536e-01,  1.4807e-01,  4.4176e-01,  5.7220e-02,  8.0888e-02],
         [-4.6173e-02, -5.4760e-01, -2.7183e-01, -9.5765e-02, -1.1808e-01,
          -1.4308e-02, -2.2081e-02,  9.6968e-02, -1.1652e-01, -1.0913e-01],
         [-3.0144e-01,  1.1947e-01, -2.1909e-02,  1.1592e-01,  2.1692e-01,
          -3.1528e-01,  4.1128e-02, -2.2072e-02, -8.7617e-02, -7.3594e-02],
         [ 2.6326e-01, -1.8113e-01,  3.7

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from lassonet import LassoNetRegressorCV

# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic data
n_samples = 1000
n_features = 10
X = np.random.randn(n_samples, n_features)
y = np.sin(X[:, 0]) + (X[:, 1])**2 + 0.1 * np.random.randn(n_samples)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)




In [2]:
# Train LassoNet with cross-validation
model = LassoNetRegressorCV(hidden_dims=(100,))
model.fit(X_train_scaled, y_train)

# Get the best lambda and selected features
best_lambda = model.best_lambda_
selected_features = model.best_selected_
print(f"Best lambda: {best_lambda}")
print(f"Selected features: {selected_features}")
print(f"Number of selected features: {len(selected_features)}")


Choosing lambda with cross-validation: 100%|██████████| 5/5 [06:08<00:00, 73.60s/it]


Best lambda: 120.41516791828191
Selected features: tensor([ True,  True, False, False, False,  True, False, False, False, False])
Number of selected features: 10


In [3]:
# Evaluate the model on the test set
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f"Test Mean Squared Error: {mse}")

Test Mean Squared Error: 0.27680444451714153


In [2]:
model2 = LassoNetRegressorCV(hidden_dims=(50,25, 10))
model2.fit(X_train_scaled, y_train)

# Get the best lambda and selected features
best_lambda = model2.best_lambda_
selected_features = model2.best_selected_
print(f"Best lambda: {best_lambda}")
print(f"Selected features: {selected_features}")
print(f"Number of selected features: {len(selected_features)}")

Choosing lambda with cross-validation: 100%|██████████| 5/5 [06:00<00:00, 72.12s/it]


Best lambda: 56.73495107381489
Selected features: tensor([ True,  True, False, False, False,  True,  True, False, False, False])
Number of selected features: 10


In [3]:
selected_features = model2.best_selected_
print(np.where(selected_features==True)[0])

[0 1 5 6]


: 

In [9]:
# Evaluate the model on the test set
y_pred = model2.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f"Test Mean Squared Error: {mse}")

Test Mean Squared Error: 0.10344699983866111


In [10]:
model5 = LassoNetRegressorCV(hidden_dims=(50,25))
model5.fit(X_train_scaled, y_train)

# Get the best lambda and selected features
best_lambda = model5.best_lambda_
selected_features = model2.best_selected_
print(f"Best lambda: {best_lambda}")
print(f"Selected features: {selected_features}")
print(f"Number of selected features: {len(selected_features)}")

Choosing lambda with cross-validation: 100%|██████████| 5/5 [05:48<00:00, 69.65s/it]


Best lambda: 56.73495107381489
Selected features: tensor([ True,  True, False, False, False, False, False, False, False, False])
Number of selected features: 10


In [11]:
# Evaluate the model on the test set
y_pred = model5.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f"Test Mean Squared Error: {mse}")

Test Mean Squared Error: 0.15267845652950218


In [6]:
model3 = LassoNetRegressorCV(hidden_dims=(10,5, 2))
model3.fit(X_train_scaled, y_train)

# Get the best lambda and selected features
best_lambda = model3.best_lambda_
selected_features = model3.best_selected_
print(f"Best lambda: {best_lambda}")
print(f"Selected features: {selected_features}")
print(f"Number of selected features: {len(selected_features)}")

Choosing lambda with cross-validation: 100%|██████████| 5/5 [05:16<00:00, 63.26s/it]


Best lambda: 34.57406583585673
Selected features: tensor([ True, False, False, False, False, False, False, False, False, False])
Number of selected features: 10


In [7]:
model3 = LassoNetRegressorCV(hidden_dims=(20,10, 5))
model3.fit(X_train_scaled, y_train)

# Get the best lambda and selected features
best_lambda = model3.best_lambda_
selected_features = model3.best_selected_
print(f"Best lambda: {best_lambda}")
print(f"Selected features: {selected_features}")
print(f"Number of selected features: {len(selected_features)}")

Choosing lambda with cross-validation: 100%|██████████| 5/5 [06:43<00:00, 80.71s/it]


Best lambda: 24.694230788901304
Selected features: tensor([ True,  True, False, False, False,  True,  True, False, False, False])
Number of selected features: 10
